In [1]:
# Download necessary NLTK resources
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pandas as pd
import json
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        print(file_path)


/kaggle/input/vietnamese-online-news-dataset/news_dataset.json


In [3]:
# Load dataset
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.json_normalize(data)

df.head()

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,218267,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,218266,HẢI YẾN - MINH LÝ,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,12,0,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170


## 1. Normalization

In [4]:
def normalize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and special characters, keeping only letters and numbers
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text
df['clean_content'] = df['content'].apply(normalize_text)
df[['content', 'clean_content']]

In [8]:
df[['content', 'clean_content']]

,content,clean_content
0,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",chiều 317 công an tỉnh thừa thiên huế đã có th...
1,"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",gần đây thứ trưởng bộ phát triển kỹ thuật số t...
2,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...
3,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,thống đốc kentucky andy beshear hôm 317 cho ha...
4,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,vụ tai nạn giao thông liên hoàn trên phố đi bộ...
...,...,...
184534,"Chiều 7/6, Khoa bị Công an quận Phú Nhuận phối...",chiều 76 khoa bị công an quận phú nhuận phối h...
184535,"Ngày 7/6, phiên xử phúc thẩm ông Tất Thành Can...",ngày 76 phiên xử phúc thẩm ông tất thành cang ...
184536,"""Các khu dân cư của thành phố Severodonetsk đã...",các khu dân cư của thành phố severodonetsk đã ...
184537,"*Việt Nam - Malaysia: 20h thứ Tư 8/6, trên VnE...",việt nam malaysia 20h thứ tư 86 trên vnexpress...


## 2. Lemmatization 

Due to each Vietnamese word does not have different variations, we can't use this technique for this data.

## 3. Tokenization

In [9]:
df['word_tokens'] = df['clean_content'].apply(word_tokenize)
df[['content','clean_content','word_tokens']]

,content,clean_content,word_tokens
0,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",chiều 317 công an tỉnh thừa thiên huế đã có th...,"[chiều, 317, công, an, tỉnh, thừa, thiên, huế,..."
1,"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",gần đây thứ trưởng bộ phát triển kỹ thuật số t...,"[gần, đây, thứ, trưởng, bộ, phát, triển, kỹ, t..."
2,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...,"[kết, quả, thi, tốt, nghiệp, thpt, năm, 2022, ..."
3,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,thống đốc kentucky andy beshear hôm 317 cho ha...,"[thống, đốc, kentucky, andy, beshear, hôm, 317..."
4,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,vụ tai nạn giao thông liên hoàn trên phố đi bộ...,"[vụ, tai, nạn, giao, thông, liên, hoàn, trên, ..."
...,...,...,...
184534,"Chiều 7/6, Khoa bị Công an quận Phú Nhuận phối...",chiều 76 khoa bị công an quận phú nhuận phối h...,"[chiều, 76, khoa, bị, công, an, quận, phú, nhu..."
184535,"Ngày 7/6, phiên xử phúc thẩm ông Tất Thành Can...",ngày 76 phiên xử phúc thẩm ông tất thành cang ...,"[ngày, 76, phiên, xử, phúc, thẩm, ông, tất, th..."
184536,"""Các khu dân cư của thành phố Severodonetsk đã...",các khu dân cư của thành phố severodonetsk đã ...,"[các, khu, dân, cư, của, thành, phố, severodon..."
184537,"*Việt Nam - Malaysia: 20h thứ Tư 8/6, trên VnE...",việt nam malaysia 20h thứ tư 86 trên vnexpress...,"[việt, nam, malaysia, 20h, thứ, tư, 86, trên, ..."


## 4. Vectorization

In [12]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['clean_content'].fillna(''))

# View the shape of the matrix
print(tfidf_matrix.shape)

(184539, 154282)


## 5. Stopwords Removal

In [ ]:
stop_words = set(stopwords.words('vietnamese'))

def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

df['cleaned_content'] = df['word_tokens'].apply(remove_stopwords)
df[['content','cleaned_content']]